In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from bs4 import BeautifulSoup
import h3
from shapely.geometry import Polygon

ModuleNotFoundError: No module named 'geopandas'

In [3]:
import boto3
from botocore.exceptions import NoCredentialsError

l3_file_list = ['private_property_facilities.parquet', 'property_listing_sales.parquet',
                'property_nearby_facilities.parquet', 'property_transactions_sales.parquet', 'property.parquet']
l3_path = '../data/L3/'

# Initialize a session using Amazon S3
s3 = boto3.client('s3')

# Replace with your details
bucket_name = 'public-git-data'
path_name = 'public/'

# Upload files to S3
try:
    for file_name in l3_file_list:
        s3.upload_file(l3_path + file_name, bucket_name, path_name + file_name)
        print(f"Upload Successful: {file_name}")
except FileNotFoundError:
    print("The file was not found")
except NoCredentialsError:
    print("Credentials not available")

Upload Successful: private_property_facilities.parquet
Upload Successful: property_listing_sales.parquet
Upload Successful: property_nearby_facilities.parquet
Upload Successful: property_transactions_sales.parquet
Upload Successful: property.parquet


In [5]:
for file_name in l3_file_list:
    print(f"https://{bucket_name}.s3.amazonaws.com/{path_name}{file_name}")

https://public-git-data.s3.amazonaws.com/public/private_property_facilities.parquet
https://public-git-data.s3.amazonaws.com/public/property_listing_sales.parquet
https://public-git-data.s3.amazonaws.com/public/property_nearby_facilities.parquet
https://public-git-data.s3.amazonaws.com/public/property_transactions_sales.parquet
https://public-git-data.s3.amazonaws.com/public/property.parquet


In [ ]:
d1= pd.read_parquet('https://public-git-data.s3.amazonaws.com/public/private_property_facilities.parquet")